Goal: Predicting the outcome of games


Step 1: Finding key variables

Dean Oliver's - FG%, TO, OR, FTA

the player should:

Shoot a high field goal percentage or help teammates shoot a high percentage through assists and playmaking.
Avoid turnovers.
Get offensive rebounds.
Draw fouls.
At the defensive end, the player should:

Make offensive players take low percentage shots.
Cause turnovers.
Box out and prevent offensive rebounds.
Avoid fouls.

Step 2: Create a model that helps us predict a game.

Create a metric that gives us how a player impacts winning based on Dean Oliver.

Given pregame lineups, determine who will win a game.

In [71]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statistics import median

%matplotlib inline

In [129]:
from datetime import datetime, timedelta
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [255]:
teamBoxScore = pandas.read_csv("nba-enhanced-stats/2012-18_teamBoxScore.csv")
teamResult = teamBoxScore['teamRslt']
teamBoxScore.head()

,gmDate,gmTime,seasTyp,offLNm1,offFNm1,offLNm2,offFNm2,offLNm3,offFNm3,teamAbbr,...,opptFIC40,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,poss,pace
0,2012-10-30,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,WAS,...,61.6667,105.6882,94.4447,11.2435,0.4390,16.7072,1.0476,33.3333,88.9409,88.9409
1,2012-10-30,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,CLE,...,56.0417,94.4447,105.6882,-11.2435,0.3765,18.8679,2.0000,84.6154,88.9409,88.9409
2,2012-10-30,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,BOS,...,80.8333,126.3381,112.6515,13.6866,0.5244,19.8287,3.1250,100.0000,94.9832,94.9832
3,2012-10-30,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,MIA,...,62.7083,112.6515,126.3381,-13.6866,0.4643,18.8501,1.5000,25.0000,94.9832,94.9832
4,2012-10-30,22:30,Regular,Foster,Scott,Zielinski,Gary,Dalen,Eric,DAL,...,58.6458,99.3678,108.1034,-8.7356,0.5000,18.6567,1.7143,42.8571,91.5790,91.5790


In [213]:
standings = pandas.read_csv("nba-enhanced-stats/2012-18_standings.csv")
standings.head()

,stDate,teamAbbr,rank,rankOrd,gameWon,gameLost,stk,stkType,stkTot,gameBack,...,rel%Indx,mov,srs,pw%,pyth%13.91,wpyth13.91,lpyth13.91,pyth%16.5,wpyth16.5,lpyth16.5
0,2012-10-30,ATL,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
1,2012-10-30,BKN,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
2,2012-10-30,BOS,14,14th,0,1,L1,loss,1,1.0,...,0.0,-13.0,-13.0,0.072,0.1687,13.8334,68.1666,0.131,10.742,71.258
3,2012-10-30,CHA,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
4,2012-10-30,CHI,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000


In [214]:
# Set some variables to hold test, dev, and training data.
test_data, test_labels = teamBoxScore[12500:], teamResult[12500:]
dev_data, dev_labels = teamBoxScore[11000:12500], teamResult[11000:12500]
train_data, train_labels = teamBoxScore[:11000], teamResult[:11000]
mini_train_data, mini_train_labels = teamBoxScore[:1000], teamResult[:1000]

In [215]:
def x_days_before(s, x):
    date = datetime.strptime(s, "%Y-%m-%d")
    modified_date = date - timedelta(days=x)
    return modified_date

In [253]:
#Naive Model - predicts team with better record.. if teams are tied, then predicts home team

#for every game in dev_data:
#  if there exists day_before record of both teams:
#    set record1, record2 to be the day_before_record of teams
#  if record1 != record2:
#    predict better record
#  else:
#    predict home team


temp_standings = standings[['stDate', 'teamAbbr', 'gameWon', 'gameLost']]
correct = 0
wrong = 0

for row in teamBoxScore.iterrows():
    date = row[1]['gmDate'] #30
    day_before = x_days_before(date, 1) #29
    first_date = datetime.strptime(temp_standings.iloc[0]['stDate'], "%Y-%m-%d") #30
    
    if day_before > first_date:
        for i in range(30):
            temp_standings = temp_standings.drop(temp_standings.index[0])
    
    team1 = row[1]['teamAbbr']
    team2 = row[1]['opptAbbr']

    if day_before == first_date:
        #FIX THIS
        dayb4 = datetime.strftime(day_before, "%Y-%m-%d")
        dayb4_df = temp_standings.loc[temp_standings['stDate'] == dayb4]
        
        team1Rec = dayb4_df.loc[dayb4_df['teamAbbr'] == team1]
        team2Rec = dayb4_df.loc[dayb4_df['teamAbbr'] == team2]
        
        team1Over500 = team1Rec.iloc[0]['gameWon'] - team1Rec.iloc[0]['gameLost']
        team2Over500 = team2Rec.iloc[0]['gameWon'] - team2Rec.iloc[0]['gameLost']
        
        if team1Over500 > team2Over500:
            prediction = 'Win'
        elif team1Over500 < team2Over500:
            prediction = 'Loss'
        elif row[1]['teamLoc'] == 'Home':
            prediction = 'Win'
        else:
            prediction = 'Loss'
    else:
        if row[1]['teamLoc'] == 'Home':
            prediction = 'Win'
        else:
            prediction = 'Loss'
    if prediction == row[1]['teamRslt']:
        correct += 1
    else:
        wrong += 1

print(correct)
print(wrong)

9549
5209


Naive model has an accuracy of ~64.7%. We should aim for our model to get above this mark.

In [256]:
deanBoxScore = teamBoxScore[['teamFG%', 'teamTO', 'teamORB', 'teamPF'
                             , 'opptFG%', 'opptTO', 'opptORB', 'opptPF']]
deanBoxScore.head()

deanResult = teamBoxScore['teamRslt']

deanResult.head()

0    Loss
1     Win
2    Loss
3     Win
4     Win
Name: teamRslt, dtype: object

In [257]:
# Set some variables to hold test, dev, and training data.
test_data1, test_labels1 = deanBoxScore[12500:], deanResult[12500:]
dev_data1, dev_labels1 = deanBoxScore[11000:12500], deanResult[11000:12500]
train_data1, train_labels1 = deanBoxScore[:11000], deanResult[:11000]
mini_train_data1, mini_train_labels1 = deanBoxScore[:1000], deanResult[:1000]

In [262]:
#KNN
for k in range(1, 20):
    model = KNeighborsClassifier(n_neighbors = k)
    model.fit(mini_train_data1, mini_train_labels1)
    print("The accuracy of KNN with " + str(k) + " neighbors is " + str(model.score(dev_data1, dev_labels1)))


The accuracy of KNN with 1 neighbors is 0.5453333333333333
The accuracy of KNN with 2 neighbors is 0.55
The accuracy of KNN with 3 neighbors is 0.5586666666666666
The accuracy of KNN with 4 neighbors is 0.5626666666666666
The accuracy of KNN with 5 neighbors is 0.5786666666666667
The accuracy of KNN with 6 neighbors is 0.5793333333333334
The accuracy of KNN with 7 neighbors is 0.5973333333333334
The accuracy of KNN with 8 neighbors is 0.584
The accuracy of KNN with 9 neighbors is 0.5786666666666667
The accuracy of KNN with 10 neighbors is 0.5713333333333334
The accuracy of KNN with 11 neighbors is 0.5786666666666667
The accuracy of KNN with 12 neighbors is 0.588
The accuracy of KNN with 13 neighbors is 0.5933333333333334
The accuracy of KNN with 14 neighbors is 0.594
The accuracy of KNN with 15 neighbors is 0.5973333333333334
The accuracy of KNN with 16 neighbors is 0.5926666666666667
The accuracy of KNN with 17 neighbors is 0.5946666666666667
The accuracy of KNN with 18 neighbors is 0

In [260]:
#NaiveBayes
model = BernoulliNB(binarize = 0.5)
model.fit(mini_train_data1, mini_train_labels1)
predicted_labels = model.predict(dev_data1)
print("The accuracy of Bernoulli's Naive Bayes Model is: " + 
      str(model.score(dev_data1, dev_labels1)))

The accuracy of Bernoulli's Naive Bayes Model is: 0.6313333333333333
